In [1]:
import requests
import json
import urlparse
import logging
import pandas as pd
import sys
import datetime
from IPython.display import display
import redis
import os

In [2]:
class TLMtkDataRemote:
    def __init__(self,token):
        self.token=token
        self.root_url="https://api.wmcloud.com:443/data/v1/"
        self._initfuncmap()
    
    def _initfuncmap(self):
        self.funcmap={"getMktEqud":"api/market/getMktEqud.json"}
        
    def _getHttp(self,url,payload):
        head_auth = {"Authorization": "Bearer "+self.token}
        try:
            r=requests.get(url,params=payload,headers=head_auth,timeout=60)
        except:
            logging.error(url+" timeout")   
            return None
        return r
        #payload = {'key1': 'value1', 'key2': 'value2'}
    
    def getMktEqud(self,secID="",ticker="",tradeDate="",field=""):
        """request
        secID	string	3选1	证券内部编码，可通过交易代码在getSecID获取到。（可多值输入，最多输入50只）
        ticker	string	3选1	股票交易代码，如'000001'（可多值输入，最多输入50只）
        tradeDate	Date	3选1	输入一个日期，不输入其他请求参数，可获取到一天全部沪深股票日行情数据，输入格式“YYYYMMDD”
        beginDate	Date	否	起始日期，输入格式“YYYYMMDD”
        endDate	Date	否	截止日期，输入格式“YYYYMMDD”
        isOpen	Int32	否	股票今日是否开盘标记位：0-今日未开盘，1-今日有开盘
        field	string	是	可选参数，用","分隔,默认为空，返回全部字段，不选即为默认值。返回字段见下方
        """
        if (secID=="" and ticker=="" and tradeDate==""):
            raise ValueError("secID,ticker,tradeDate 至少填写一个")
        
        req_params={"secID":secID,"ticker":ticker,"tradeDate":tradeDate,"field":field}
        
        desturl=urlparse.urljoin(self.root_url,self.funcmap["getMktEqud"])
        
        #print u"请求地址:",desturl
        
        r=self._getHttp(desturl,req_params)
        
        while (r==None):
            loging.info("wait 120s")
            time.sleep(120)
            r=self._getHttp(desturl,req_params)
        
        if (r.status_code!=200):
            logging.error("req "+r.url+r.status_code)
            return None
        
        
        jdata= r.json()
        if (jdata.get("data")==None):    
            if (jdata.get("retCode")=="-1"):
                logging.info(tradeDate+json.dumps(jdata))
                return {"data":[]}
            else:
                logging.error(tradeDate+json.dumps(jdata))      
                return None      
        return jdata
        """response
        secID	String	证券内部编码
        tradeDate	Date	交易日
        ticker	string	证券代码
        secShortName	string	证券简称
        exchangeCD	string	交易所代码
        preClosePrice	Double	昨收盘
        actPreClosePrice	Double	实际昨收盘
        openPrice	Double	今开盘
        highestPrice	Double	最高价
        lowestPrice	Double	最低价
        closePrice	Double	今收盘
        turnoverVol	Double	成交量
        turnoverValue	Double	成交金额
        dealAmount	Int32	成交笔数
        turnoverRate	Double	日换手率
        accumAdjFactor	Double	累积前复权因子，前复权是对历史行情进行调整，除权除息日的行情不受本次除权除息影响，不需进行调整。最新一次除权除息日至最新行情期间的价格不需要进行任何的调整，该期间前复权因子都是1。
        negMarketValue	Double	流通市值
        marketValue	Double	总市值
        PE	Double	滚动市盈率,将于2016-01-01停止维护，推荐使用/api/market/getStockFactorsOneDay或/api/market/getStockFactorsDateRange，返回字段PE
        PE1	Double	市盈率，将于2016-01-01停止维护，推荐使用/api/market/getStockFactorsOneDay或/api/market/getStockFactorsDateRange
        PB	Double	市净率，将于2016-01-01停止维护，推荐使用/api/market/getStockFactorsOneDay或/api/market/getStockFactorsDateRange，返回字段PB
        isOpen	Int32	是否开盘
        """  

In [12]:
class TLMtkDataLocal:
    def __init__(self,filepath):
        self._filepath=filepath
        self._mtkdataPre="mtkdata"
        self._mtkday="mtkday"
        #self._store = pd.HDFStore('/data/notebooks/mtkdata/store_day.h5')
        self._store=pd.HDFStore(filepath)
        
    def __del__(self):
        if self._store.isopen()==True:
            self._store.close()
    
    #检查并创建表
    def _checkAndCreateTable(self):
        print 11
        #if ("/"+self._mtkday) not in self._store.keys():    
            #df=pd.DataFrame(columns=['day','state']) 
            #self._store.put(self._mtkday,df,append=true,format="table")
          
            
    def _getTbname(self,daystr):
        return self._mtkday+"_"+daystr
    
    #设置日数据
    def setData(self,daystr,df):     
        self._checkAndCreateTable()
        tb=self._getTbname(daystr)
        state='open';
        if df.empty:
            state="close"
           
        self._store.put(tb,df)  
        
        upddf=pd.DataFrame.from_dict([{"day":daystr,"state":state}]).set_index(['day'])
        
        if ("/"+self._mtkday) not in self._store.keys(): 
            self._store.append(self._mtkday,upddf)
        #检查数据
        fd=self._store.select(self._mtkday,where=["day='"+daystr+"'"])
            
        if fd.empty: 
            self._store.append(self._mtkday,upddf)
        else:
            self._store[self._mtkday].update(upddf)

    #删除日数据
    def removeData(self,daystr):
        self._checkAndCreateTable()
        tb=self._getTbname(daystr)
        if ("/"+tb) in self._store.keys():
            del store[tb]
            md=store[self._mtkday]
            md.drop[md.index.get_level_values('day')==daystr]
            store[self._mtkday]=md

    def hasData(self,daystr):
        self._checkAndCreateTable()
        tb=self._getTbname(daystr)
        if ("/"+self._mtkday) not in self._store.keys(): 
            return "not find"
        
        fd=self._store.select(self._mtkday,where=["day='"+daystr+"'"])
        
        if fd.empty:
            return "not find";
        
        return fd.iloc[0,1]
    
    def getData(self,daystr):
        self._checkAndCreateTable()
        tb=self._getTbname(daystr)
        checkresult=self.hasData(daystr)
        if ( checkresult=="not find" or checkresult=="error"):
            return None
        
        if (checkresult=="close"):
            return pd.DataFrame(columns=[])
        
        return self._store[tb]
        
        '''
        tname="mtkdate"
        daylinestr=datetime.datetime.strftime(datetime.datetime.strptime(daystr, "%Y%m%d"),'%Y-%m-%d')
        if ("/"+tname) in store.keys():
            fd=store.select("mtkdate",where=["tradeDate=='"+daylinestr+"'"])
            #print daylinestr,len(fd)
            #print len(fd)
            if len(fd)==0:
                return False
            else:
                return True
        else:
            return False
            '''

In [13]:
class TLMtkData:
    def __init__(self,filepath,token):
        self._local=TLMtkDataLocal(filepath)
        self._remote=TLMtkDataRemote(token)
    
    def getMtkDataByDay(self,daystr):
        result=self._local.getData(daystr)
        if result==None:
            rsp=self._remote.getMktEqud(tradeDate=daystr)
            if (rsp!=None):
                result=pd.DataFrame(rsp["data"])
                #解决unicode转换
                types = result.apply(lambda x: pd.lib.infer_dtype(x.values))
                for col in types[types=='unicode'].index:
                    result[col] = [c.encode('utf-8') for c in result[col].values]

                #print types

                #解决类型猜测错误问题
                for col in types[types=='integer'].index:
                    if col not in ['isOpen']:
                        #print col
                        result[col] = result[col].astype(float)

                result.columns = [str(c) for c in result.columns]

                result=result.set_index(['tradeDate','secID'])
                self._local.setData(daystr,result)
        return result

In [14]:
if __name__=="__main__":
    log_format = "%(filename)s [%(asctime)s] [%(levelname)s] %(message)s"
    logging.basicConfig(format=log_format,datefmt='%Y-%m-%d %H:%M:%S %p',level=logging.DEBUG)
    
    # 定义一个Handler打印Debug及以上级别的日志到sys.stderr  
    console = logging.StreamHandler()  
    console.setLevel(logging.DEBUG)  
    # 设置日志打印格式  
    formatter = logging.Formatter('%(name)-12s: %(levelname)-8s %(message)s') 
    console.setFormatter(formatter)  
    # 将定义好的console日志handler添加到root logger  
    logging.getLogger("").addHandler(console)  
    
    #定义redis链接
    #redishost=os.getenv('REDIS_PORT_6379_TCP_ADDR')
    #r = redis.Redis(host=redishost,port=6379,db=0)
    
    #tk=TLMtkData(sys.argv[1])
    tk=TLMtkData("/data/notebooks/mtkdata/store_day.h5","00b8a504099fb24bb62772d1318f69abd0ab4575734770349aaeaf6e3774557d")
    
    #begin=datetime.datetime.strptime(sys.argv[1], "%Y%m%d")   
    #end=datetime.datetime.strptime(sys.argv[2], "%Y%m%d")
    begin = datetime.date(2007, 1, 1)  
    end = datetime.date(2016,4,24)

    for i in range((end - begin).days+1):  
        day = begin + datetime.timedelta(days=i)
        daystr=datetime.datetime.strftime(day,'%Y%m%d')    
        df=tk.getMtkDataByDay(daystr)
        print df
        
    

root        : ERROR    20070101{"retCode": -1, "retMsg": "No Data Returned"}
root        : ERROR    20070101{"retCode": -1, "retMsg": "No Data Returned"}
root        : ERROR    20070101{"retCode": -1, "retMsg": "No Data Returned"}
root        : ERROR    20070101{"retCode": -1, "retMsg": "No Data Returned"}


11
11
None
11
11


root        : ERROR    20070102{"retCode": -1, "retMsg": "No Data Returned"}
root        : ERROR    20070102{"retCode": -1, "retMsg": "No Data Returned"}
root        : ERROR    20070102{"retCode": -1, "retMsg": "No Data Returned"}
root        : ERROR    20070102{"retCode": -1, "retMsg": "No Data Returned"}
root        : ERROR    20070103{"retCode": -1, "retMsg": "No Data Returned"}
root        : ERROR    20070103{"retCode": -1, "retMsg": "No Data Returned"}
root        : ERROR    20070103{"retCode": -1, "retMsg": "No Data Returned"}
root        : ERROR    20070103{"retCode": -1, "retMsg": "No Data Returned"}


None
11
11
None
11
11
11


Exception AttributeError: AttributeError("'HDFStore' object has no attribute 'isopen'",) in <bound method TLMtkDataLocal.__del__ of <__main__.TLMtkDataLocal instance at 0x7ffa2dd39320>> ignored


ValueError: The passed where expression: [day='20070104']
            contains an invalid variable reference
            all of the variable refrences must be a reference to
            an axis (e.g. 'index' or 'columns'), or a data_column
            The currently defined references are: index,columns
